In [ ]:
# default_exp scrape13G

# scrape13G

> Scrape holdings information from 13G SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default13GDir = os.path.join(utils.stockDataRoot,'scraped13G')

13G scraper class - scrape holdings information from the SEC filing:

In [ ]:
#export

def parse13G(accNo, formType=None) :
    info = basicInfo.getSecFormInfo(accNo, formType=formType, get99=True, textLimit=textLimit)
    links = info['links']
    if len(links) == 0 :
        utils.printErrInfoOrAccessNo('NO LINKS LIST in',accNo)
        return info
    toFormat = 'text' if links[0][3].endswith('.txt') else 'souptext'
    mainText = utils.downloadSecUrl(links[0][3], toFormat=toFormat)
    return info, mainText

class scraper13G(infoScraper.scraperBase) :
    def __init__(self, infoDir=default13GDir, startD=None, endD=None, fSuff='m.pkl', **pickle_kwargs) :
        super().__init__(infoDir, 'SC 13G', startD=startD, endD=endD, fSuff=fSuff, **pickle_kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse13G(accNo, formType), None

Test 13G scraper class:

In [ ]:
dl = dailyList.dailyList(startD='empty')
dl.updateForDays('20210701','20210704')
assert len(dl.getFilingsList(None,'SC 13G')[0])==100,"testing 13G scraper class (daily list count)"
# info = parse13G('0001165002-21-000068', formType='SC 13G', textLimit=1000)
# assert (info['itemTexts'][0].startswith('ITEM 2.02: RESULTS OF OPERATIONS AND FINANCIAL CONDITION '
#                                         +'On July 27, 2021 , Westwood')
#         and info['itemTexts'][0].endswith('otherwise expressly stated in such filing. ')
#         and info['itemTexts'][1].startswith('ITEM 7.01: REGULATION FD DISCLOSURE Westwood')
#         and info['itemTexts'][1].endswith('of record on August 6, 2021. ')
#         and info['itemTexts'][2].startswith('ITEM 9.01: FINANCIAL STATEMENTS AND EXHIBITS (d) ')
#         and info['itemTexts'][2].endswith('Financial Officer and Treasurer')
#         and info['text99'][1].startswith('EX-99.1 2 a2q21earningsrelease.htm EX-99.1 Document Westwood')
#         and info['text99'][1].endswith('Income Opportunity and Total R')
#     ),"testing 8-K scraper class (parsing)"

20210703 WEEKEND 20210702 ### list index 21 filings for 20210702: 6569 * 20210701 filings for 20210701: 5573 * 

In [ ]:
#hide
# assorted test code: 

# dl = dailyList.dailyList(startD='20200101')

# s = scraper6K(startD='empty')
# s.updateForDays(dl,startD='20210719',endD='20210721')
# print(); s.printCounts()

# dailyList.dlCountFilings(startD='20210726',endD='20210731',formClass='13F-HR',noAmend=False)

# accNo = '0001174947-20-001195'
# utils.secBrowse(accNo)
# i = parse8K(accNo,'8-K')
# i

# b = utils.downloadSecUrl('/Archives/edgar/data/80172/000143774920015853/npk20200727_8k.htm',toFormat='souptext')

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()